<a href="https://colab.research.google.com/github/zjubirdfly/WebCola/blob/master/GeneratePicture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Connect to google cloud and import data
import uuid
import pandas as pd
import numpy as np
import os
import glob
import shutil
from google.colab import auth
from google.colab import drive
project_id = 'directed-symbol-252319'
bucket_name = "chongqingtaxidata"
raw_data_bucket_name = "raw_taxi_data"
destination_path = "/content/Taxidata/"
auth.authenticate_user()
drive.mount('/content/drive')
!gcloud config set project {project_id}

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Updated property [core/project].


In [0]:
#Copy files into system
!mkdir {destination_path}
!gsutil -m cp -r gs://{raw_data_bucket_name}/201703/25 {destination_path}


In [0]:
data_cols = ['date','time', 'label', 'tag','lat','lon','speed?','direction?','status', '?']
files = glob.glob("/content/Taxidata/25/20170325_287.TXT")

all_date_data = []
sample_data = []
for file in files:
    sample_data.append(pd.read_csv(file, sep=',',encoding='GB2312', names= data_cols, header=None,  dtype={'time': object}))

all_date_data = pd.concat(sample_data, axis=0, ignore_index=True)

In [6]:
# Generate N*N graph
N = 25

max_lat = all_date_data['lat'].max()
min_lat = all_date_data[all_date_data['lat'] > 0.1]['lat'].min()
lat_range = (max_lat - min_lat) / N

max_lon = all_date_data['lon'].max()
min_lon = all_date_data[all_date_data['lon'] > 0.1]['lon'].min()
lon_range = (max_lon - min_lon) / N

graph = np.zeros((N,N))

for index,row in all_date_data.iterrows(): 
  lat = row['lat']
  lon = row['lon']
  if lat == 0 and lon == 0:
    continue
  print(lat, lon)

  #lat index
  lat_index = 0;
  if lat < min_lat:
    lat_index = 0
  elif lat > max_lat:
    lat_index = N - 1
  else:
    lat_index = int((lat - min_lat) / lat_range)

  #lon index
  lon_index = 0;
  if lon < min_lon:
    lon_index = 0
  elif lon > max_lon:
    lon_index = N - 1
  else:
    lon_index = int((lon - min_lon) / lon_range)

  graph[lat_index, lon_index] = graph[lat_index, lon_index] + 1


106.542755 29.57893
106.49388700000002 29.620387
106.542755 29.57893
106.650897 29.036141999999998
106.552977 29.500718
106.49388700000002 29.620387
106.51301799999999 29.592707
106.542755 29.57893
106.650897 29.036141999999998
106.50973700000002 29.586978000000002
106.552977 29.500718
106.49388700000002 29.620387
106.51301799999999 29.592707
106.49341499999998 29.62068
106.542755 29.57893
106.559503 29.627834999999997
106.650897 29.036141999999998
106.47896999999999 29.537805
106.50973700000002 29.586978000000002
106.613155 29.698738
106.552977 29.500718
106.51671 29.58724
106.49388700000002 29.620387
106.51301799999999 29.592707
106.537948 29.608247
106.49341499999998 29.62068
106.509023 29.592625
106.55591499999998 29.516978
106.559503 29.627834999999997
106.542755 29.57893
106.5729 29.499846999999995
106.650897 29.036141999999998
106.47896999999999 29.537805
106.50973700000002 29.586978000000002
106.552977 29.500718
106.51671 29.58724
106.49388700000002 29.620387
106.51301799999999

KeyboardInterrupt: ignored

In [37]:
graph

array([[1.83000e+02, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00],
       [0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00],
       [1.30000e+01, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 0.00000e+00, 0.